In [20]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D, BatchNormalization, Activation, InputLayer
import gc


In [21]:

# Dataset directory
dataset_dir = 'dataset/'

# Define the target shape for images
TARGET_SHAPE = (128, 128, 3)


In [22]:

# Data sampling: Select 10-20% of the dataset
sampling_fraction = 0.15  # Adjust this between 0.1 and 0.2 for 10-20% sampling
file_list = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.npz')]
sampled_file_list = random.sample(file_list, int(len(file_list) * sampling_fraction))
print(f"Number of files sampled: {len(sampled_file_list)}")


Number of files sampled: 329


In [23]:

# Resize function
def resize_image(image):
    return resize(image, TARGET_SHAPE, preserve_range=True, anti_aliasing=True)

# Function to load .npz files using memory mapping
def load_npz_file(file_path):
    data = np.load(file_path, mmap_mode='r')  # Memory-mapped to avoid loading entirely into RAM
    color_images = data['colorImages'] / 255.0  # Normalize images
    color_images_resized = np.array([resize_image(img) for img in color_images.transpose(3, 0, 1, 2)])
    
    bounding_boxes = data['boundingBox']
    landmarks_2d = data['landmarks2D']
    landmarks_3d = data['landmarks3D']
    
    gc.collect()  # Free memory after loading
    return color_images_resized, bounding_boxes, landmarks_2d, landmarks_3d


In [24]:

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator with reduced batch size and single-threaded processing
def data_generator(file_list, batch_size=2):
    while True:
        for i in range(0, len(file_list), batch_size):
            batch_files = file_list[i:i + batch_size]
            images = []
            
            with ThreadPoolExecutor(max_workers=1) as executor:  # Reduced to single thread for stability
                results = list(tqdm(executor.map(load_npz_file, batch_files), total=len(batch_files), desc="Loading batch"))
            
            for result in results:
                color_images, _, _, _ = result
                images.append(color_images)
            
            images = np.concatenate(images, axis=0)
            augmented_images = np.array([datagen.random_transform(img) for img in images])
            
            gc.collect()  # Manually trigger garbage collection to free memory
            
            yield augmented_images, augmented_images


In [25]:

# Split the sampled file list into training and validation sets
train_files, val_files = train_test_split(sampled_file_list, test_size=0.2, random_state=42)
gc.collect()  # Free memory after splitting data


17274

In [26]:

# Example usage of data_generator
train_gen = data_generator(train_files, batch_size=2)
val_gen = data_generator(val_files, batch_size=2)


In [30]:
# Build the Image Enhancement Model with UpSampling2D factor of 2
def build_enhancement_model(input_shape):
    model = Sequential()

    # Input Layer
    model.add(InputLayer(input_shape=input_shape))

    # First Convolution Block
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # Additional Convolution Layers for Image Enhancement
    for _ in range(4):
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))


    # Output Layer
    model.add(Conv2D(3, (3, 3), padding='same', activation='sigmoid'))  # Use sigmoid for output in range [0,1]

    return model

# Compile the model
model = build_enhancement_model(TARGET_SHAPE)
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_33 (Conv2D)              │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_30 (Activation)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_31 (Activation)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_32 (Activation)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_33 (Activation)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_34 (Activation)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 128, 128, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 152,515 (595.76 KB)

 Trainable params: 151,875 (593.26 KB)

 Non-trainable params: 640 (2.50 KB)

In [29]:

# Train the model
history = model.fit(train_gen, validation_data=val_gen, epochs=10, steps_per_epoch=50, validation_steps=10)


Loading batch: 100%|██████████| 2/2 [00:02<00:00,  1.09s/it]


Epoch 1/10
 1/50 ━━━━━━━━━━━━━━━━━━━━ 40:00 49s/step - accuracy: 0.3670 - loss: 0.0874

Loading batch: 100%|██████████| 2/2 [01:00<00:00, 30.11s/it]


 2/50 ━━━━━━━━━━━━━━━━━━━━ 1:02:59 79s/step - accuracy: 0.3945 - loss: 0.0770

Loading batch:  50%|█████     | 1/2 [00:53<00:53, 53.57s/it]

In [ ]:

# Save the trained model for future use
model.save('facial_reconstruction_model.h5')


In [ ]:

# Visualize and Compare Results: Original vs Enhanced
def plot_comparison(original, enhanced):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(original)
    plt.title("Original Image")
    
    plt.subplot(1, 2, 2)
    plt.imshow(enhanced)
    plt.title("Enhanced Image")
    plt.show()

# Generate a batch of images for evaluation
eval_batch = next(val_gen)

# Run the model on a sample batch to see the enhancement results
sample_original_images = eval_batch[0]  # Input images (low-quality)
sample_enhanced_images = model.predict(sample_original_images)

# Plot side-by-side comparison for the first image in the batch
plot_comparison(sample_original_images[0], sample_enhanced_images[0])


In [ ]:

# Display the model's performance over epochs
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Training vs Validation Loss")
plt.show()
